# Example
Comparison between swept wing by Bertin and pySailingVLM results. More information can be found at Aerodynamics for engineers John J.Bertin p 368 (Example 7.2).

In [1]:
# varaibles.py for jupyter
import os
import numpy as np
import time
from pySailingVLM.runner.container import Output, Rig, Conditions, Solver, MainSail, JibSail, Csys, Keel

half_wing_span = 0.5
sweep_angle_deg = 0.
chord_length = 0.2
AoA_deg = 4.
mgirths =  np.array([0.00, 1./8, 1./4, 1./2, 3./4, 7./8, 1.00])
mchords = np.array([chord_length]* len(mgirths))
jgirths = np.array([0.00, 1./4, 1./2, 3./4, 1.00])

out = Output(case_name='my_case_name',
             case_dir=os.path.abspath(''),
             name=os.path.join("results_example_jib_and_mainsail_vlm", time.strftime("%Y-%m-%d_%Hh%Mm%Ss")),
            file_name='my_fancy_results')

solver = Solver(n_spanwise=5,
                n_chordwise=3,
                interpolation_type='linear')

conditions = Conditions(leeway_deg=0.,    
                        heel_deg=0.,    
                        SOG_yacht=0.,  
                        tws_ref= 1.,     
                        alpha_true_wind_deg= 0., 
                        reference_water_level_for_wind_profile=-0.,
                        wind_exp_coeff=0.,
                        wind_reference_measurment_height=10.,
                        rho=1.,
                        wind_profile='flat',
                        roughness=0.05)

rig = Rig(main_sail_luff=half_wing_span / np.cos(np.deg2rad(sweep_angle_deg)),
          jib_luff=10.0,
          foretriangle_height=11.50,
          foretriangle_base=3.90,
          sheer_above_waterline=0.,
          boom_above_sheer=0.,
          rake_deg=90. + sweep_angle_deg,
          mast_LOA=0.,
          sails_def='main')

main = MainSail(centerline_twist_deg=np.array([AoA_deg] * len(mgirths)), #0*mgirths,
                girths=mgirths,
                chords=mchords,
                camber= 0*np.array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]),
                camber_distance_from_luff=np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]))
              
jib = JibSail(centerline_twist_deg=0*(10+5)  + 0*15. * jgirths,
             girths=0*jgirths,
             chords=0* np.array([3.80, 2.98, 2.15, 1.33, 0.5]),
             camber=0*np.array([0.01, 0.01, 0.01, 0.01, 0.01]),
             camber_distance_from_luff=0*np.array([0.5, 0.5, 0.5, 0.5, 0.5]))

csys = Csys(reference_level_for_moments=np.array([0, 0, 0]))
keel = Keel(center_of_lateral_resistance_upright=np.array([0, 0, -1.0]))    

wind
<div class="alert alert-block alert-warning">
<b>Run cell below twice:</b> Run cell below twice before running other cells. Output below is expected to always appear at first time importing pySailingVLM:
<br><br>
    
<em>   
/home/user/miniconda3/envs/mgr_test/lib/python3.10/site-packages/numba/core/lowering.py:107: NumbaDebugInfoWarning: Could not find source for function: <function __numba_array_expr_0x7f01d6a1e9e0 at 0x7f01d6cfa680>. Debug line information may be inaccurate.warnings.warn(NumbaDebugInfoWarning(msg))
</em> 
<br>
</div>

In [2]:
import shutil
from pySailingVLM.rotations.csys_transformations import CSYS_transformations
from pySailingVLM.yacht_geometry.hull_geometry import HullGeometry
from pySailingVLM.results.save_utils import save_results_to_file
from pySailingVLM.solver.panels_plotter import display_panels_xyz_and_winds
from pySailingVLM.results.inviscid_flow import InviscidFlowResults
from pySailingVLM.solver.vlm import Vlm
from pySailingVLM.runner.sail import Wind, Sail
from pySailingVLM.solver.panels_plotter import plot_cp

In [3]:
import numpy as np
from pySailingVLM.solver.coefs import get_vlm_Cxyz

AoA_degs = np.linspace(0.001, 10., 20) # [4.0] #

C_results = []
cl_results = []
a_vlm_results = []
# for AoA_deg in AoA_degs:
#conditions.alpha_true_wind_deg=  AoA_deg
csys_transformations = CSYS_transformations(conditions.heel_deg, conditions.leeway_deg, v_from_original_xyz_2_reference_csys_xyz=csys.reference_level_for_moments)

w = Wind(conditions)
s = Sail(solver, rig, main, jib, csys_transformations)
sail_set = s.sail_set
myvlm = Vlm(sail_set.panels, solver.n_chordwise, solver.n_spanwise, conditions.rho, w.profile, sail_set.trailing_edge_info, sail_set.leading_edge_info)

height = 1.0
sails_Cxyz = myvlm.get_Cxyz(w, height)

    # enumerate through sails
    # in this example we have only main
for idx, Cxyz in enumerate(sails_Cxyz):
    a_vlm = Cxyz[1] / np.deg2rad(AoA_deg)
    C_results.append(Cxyz[1])
    a_vlm_results.append(a_vlm)


In [4]:
# results from Aerodynamics for engineers John J.Bertin p 368
C_bertin = 1.096 * np.pi * np.pi / 180.* AoA_degs

In [5]:
# import matplotlib.pyplot as plt
# import scienceplots # sudo apt-get install dvipng texlive-latex-extra texlive-fonts-recommended cm-super

# # plt.style.use('science')
# fig = plt.figure()
# ax = fig.add_subplot(1, 1, 1)
# ax.plot(AoA_degs, C_results, color='tab:blue', label='pySailingVLM')
# ax.plot(AoA_degs, C_bertin, color='tab:orange', linestyle='dashed', label='Bertin')

# ax.set_xlabel(r"AoA [$^\circ$]")
# ax.set_ylabel(r"$C_{L}$")

# ax.set_title("Lift coefficient versus angle of attack.")
# ax.legend()
# plt.plot(AoA_degs, C_results)   
# #plt.savefig('bertin_1.png')

In [6]:
# import numpy as np
# from pySailingVLM.solver.coefs import get_vlm_Cxyz

# AoA_deg = 4.2
# C = []
# c_l = []
# a_vlm = []

# conditions.alpha_true_wind_deg = AoA_deg
# solver.n_spanwise = 4

# csys_transformations = CSYS_transformations(conditions.heel_deg, conditions.leeway_deg, v_from_original_xyz_2_reference_csys_xyz=csys.reference_level_for_moments)

# w = Wind(conditions)
# s = Sail(solver, rig, main, jib, csys_transformations)
# sail_set = s.sail_set
# myvlm = Vlm(sail_set.panels, solver.n_chordwise, solver.n_spanwise, conditions.rho, w.profile, sail_set.trailing_edge_info, sail_set.leading_edge_info)

# height = 1.0
# sails_Cxyz = myvlm.get_Cxyz(w, height)
 
# for idx, Cxyz in enumerate(sails_Cxyz):
#     a = Cxyz[1] / np.deg2rad(AoA_deg)
#     C.append(Cxyz[1])
#     a_vlm.append(a)

In [7]:
# def calculate_ydata(gamma, CL, cav, wind, height):
#     V = np.array(wind.profile.get_true_wind_speed_at_h(height))
#     return 2 * gamma / (np.linalg.norm(V) * cav * CL)

# N = int(sail_set.panels.shape[0] / 2)
# cav = chord_length
# y_data = np.array([calculate_ydata(myvlm.gamma_magnitude[i], C[0], cav, w, height) for i in range(N)])
# cp_y = np.split(myvlm.cp, 2)[0][:,2]
# x_data = 2 * cp_y / (half_wing_span * 2)

In [8]:
# g_good = np.array([0.0273, 0.0287, 0.0286, 0.0250])
# ydata_good = 2 * 4 * 2 * half_wing_span * g_good / (1.096 * cav)
# cp_y_good = np.array([0.0625, 0.1875, 0.3125, 0.4377])
# x_good = 2*cp_y_good / (2*half_wing_span)

In [9]:
# # importing matplotlib module
# from matplotlib import pyplot as plt
# fig = plt.figure()
# ax = fig.add_subplot(1, 1, 1)
# ax.plot(x_data, y_data, 'bo',label='pySailingVLM')
# ax.plot(x_good, ydata_good,'r+', label='Bertin')

# ax.set_xlabel(r"$\frac{2y}{b}$")
# ax.set_ylabel(r"$\frac{C_{l}}{C_{L}}$")
# ax.set_title("Spanwise lift distribution")
# ax.legend()   
# plt.savefig('bertin_2')

In [10]:
from pySailingVLM.solver.coefs import get_C
cl = get_C(myvlm.panels, myvlm.areas, myvlm.lift, myvlm.inlet_conditions.V_app_infs, myvlm.n_spanwise, myvlm.n_chordwise, myvlm.rho)
cd = get_C(myvlm.panels, myvlm.areas, myvlm.drag, myvlm.V_induced_at_cp, myvlm.n_spanwise, myvlm.n_chordwise, myvlm.rho)

In [11]:
#np.testing.assert_almost_equal(myvlm.inlet_conditions.V_app_infs,w.profile.get_true_wind_speed_at_h(1.0))
    

In [12]:
w.profile.get_true_wind_speed_at_h(1.0)

array([1., 0., 0.])

In [13]:
myvlm.inlet_conditions.V_app_infs

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [14]:
myvlm.force

array([[-2.37539955e-05, -2.12735109e-03, -1.64826454e-21],
       [-2.09036323e-05, -2.08830429e-03, -5.97876896e-21],
       [-1.47950654e-05, -1.99814039e-03, -5.13306947e-22],
       [-4.63387809e-06, -1.82146648e-03, -3.21539884e-22],
       [ 9.89734224e-06, -1.44248351e-03,  6.86766077e-22],
       [ 4.61873843e-05, -8.22329568e-04,  0.00000000e+00],
       [ 4.52470602e-05, -8.02376225e-04, -6.80213367e-22],
       [ 4.29981984e-05, -7.55501677e-04, -6.55293248e-21],
       [ 3.83517926e-05, -6.61522512e-04,  5.32238039e-21],
       [ 2.81110057e-05, -4.63500329e-04,  0.00000000e+00],
       [ 2.47047914e-05, -4.00656100e-04,  0.00000000e+00],
       [ 2.40714058e-05, -3.89098327e-04,  0.00000000e+00],
       [ 2.25723514e-05, -3.62039524e-04,  6.26508606e-21],
       [ 1.95741546e-05, -3.09051793e-04,  0.00000000e+00],
       [ 1.36113981e-05, -2.07926073e-04,  0.00000000e+00],
       [ 9.89734224e-06, -1.44248351e-03, -6.86766077e-22],
       [-4.63387809e-06, -1.82146648e-03

In [15]:
myvlm.lift

array([[ 0.00000000e+00, -2.12440401e-03,  0.00000000e+00],
       [ 0.00000000e+00, -2.08560246e-03, -4.52242944e-21],
       [ 0.00000000e+00, -1.99598957e-03,  0.00000000e+00],
       [ 0.00000000e+00, -1.82030537e-03,  0.00000000e+00],
       [ 0.00000000e+00, -1.44286391e-03,  0.00000000e+00],
       [ 0.00000000e+00, -8.24931899e-04,  0.00000000e+00],
       [ 0.00000000e+00, -8.04922307e-04, -6.98158811e-21],
       [ 0.00000000e+00, -7.57925083e-04, -6.57395217e-21],
       [ 0.00000000e+00, -6.63727595e-04,  0.00000000e+00],
       [ 0.00000000e+00, -4.65275172e-04,  0.00000000e+00],
       [ 0.00000000e+00, -4.01984721e-04,  0.00000000e+00],
       [ 0.00000000e+00, -3.90374614e-04,  0.00000000e+00],
       [ 0.00000000e+00, -3.63199260e-04,  0.00000000e+00],
       [ 0.00000000e+00, -3.10003642e-04,  0.00000000e+00],
       [ 0.00000000e+00, -2.08534257e-04,  0.00000000e+00],
       [ 0.00000000e+00, -1.44286391e-03,  0.00000000e+00],
       [ 0.00000000e+00, -1.82030537e-03

In [16]:
a = np.array([0.0021244 , 0.0020856 , 0.00199599, 0.00182031, 0.00144286,
       0.00082493, 0.00080492, 0.00075793, 0.00066373, 0.00046528,
       0.00040198, 0.00039037, 0.0003632 , 0.00031   , 0.00020853,
       0.00144286, 0.00182031, 0.00199599, 0.0020856 , 0.0021244 ,
       0.00046528, 0.00066373, 0.00075793, 0.00080492, 0.00082493,
       0.00020853, 0.00031   , 0.0003632 , 0.00039037, 0.00040198])
np.testing.assert_almost_equal(myvlm.force[:, 1], a, decimal=2)

In [17]:
myvlm.force[:, 1]

array([-0.00212735, -0.0020883 , -0.00199814, -0.00182147, -0.00144248,
       -0.00082233, -0.00080238, -0.0007555 , -0.00066152, -0.0004635 ,
       -0.00040066, -0.0003891 , -0.00036204, -0.00030905, -0.00020793,
       -0.00144248, -0.00182147, -0.00199814, -0.0020883 , -0.00212735,
       -0.0004635 , -0.00066152, -0.0007555 , -0.00080238, -0.00082233,
       -0.00020793, -0.00030905, -0.00036204, -0.0003891 , -0.00040066])

In [18]:
myvlm.drag

array([[-2.37539955e-05, -2.94707202e-06, -1.64826454e-21],
       [-2.09036323e-05, -2.70182535e-06, -1.45633952e-21],
       [-1.47950654e-05, -2.15082093e-06, -5.13306947e-22],
       [-4.63387809e-06, -1.16110664e-06, -3.21539884e-22],
       [ 9.89734224e-06,  3.80400650e-07,  6.86766077e-22],
       [ 4.61873843e-05,  2.60233154e-06, -0.00000000e+00],
       [ 4.52470602e-05,  2.54608200e-06,  6.30137475e-21],
       [ 4.29981984e-05,  2.42340553e-06,  2.10196923e-23],
       [ 3.83517926e-05,  2.20508292e-06,  5.32238039e-21],
       [ 2.81110057e-05,  1.77484332e-06, -0.00000000e+00],
       [ 2.47047914e-05,  1.32862107e-06, -0.00000000e+00],
       [ 2.40714058e-05,  1.27628626e-06, -0.00000000e+00],
       [ 2.25723514e-05,  1.15973575e-06,  6.26508606e-21],
       [ 1.95741546e-05,  9.51849699e-07, -0.00000000e+00],
       [ 1.36113981e-05,  6.08183903e-07, -0.00000000e+00],
       [ 9.89734224e-06,  3.80400650e-07, -6.86766077e-22],
       [-4.63387809e-06, -1.16110664e-06

In [19]:
myvlm.V_induced_at_ctrl

array([[-4.19208450e-03,  6.96336728e-02, -4.63461069e-05],
       [-4.19575172e-03,  6.96334164e-02, -1.47877596e-04],
       [-4.21733559e-03,  6.96319071e-02, -2.78353169e-04],
       [-4.30408043e-03,  6.96258413e-02, -4.61724835e-04],
       [-4.59370792e-03,  6.96055886e-02, -6.80800068e-04],
       [-4.00782355e-03,  6.96465576e-02, -3.82002885e-05],
       [-3.98397728e-03,  6.96482251e-02, -1.24953931e-04],
       [-3.94137234e-03,  6.96512043e-02, -2.49501468e-04],
       [-3.90989621e-03,  6.96534054e-02, -4.68527633e-04],
       [-4.08637265e-03,  6.96410649e-02, -9.39701319e-04],
       [-3.74951833e-03,  6.96646201e-02,  1.01712147e-04],
       [-3.67374992e-03,  6.96699183e-02,  3.36312435e-04],
       [-3.49724145e-03,  6.96822610e-02,  6.88986834e-04],
       [-3.14746413e-03,  6.97067198e-02,  1.36694721e-03],
       [-2.37286085e-03,  6.97608853e-02,  3.23570473e-03],
       [-4.59370792e-03,  6.96055886e-02,  6.80800068e-04],
       [-4.30408043e-03,  6.96258413e-02

In [20]:
myvlm.V_induced_at_cp

array([[ 1.38724649e-03, -1.11814868e-02, -4.54729121e-05],
       [ 1.29546517e-03, -1.00228268e-02, -1.43248957e-04],
       [ 1.07757123e-03, -7.41239617e-03, -2.61785074e-04],
       [ 6.37863656e-04, -2.54565973e-03, -4.10133117e-04],
       [-2.63642778e-04,  6.85951194e-03, -5.48087554e-04],
       [-3.15460166e-03,  5.59893299e-02, -4.53766789e-05],
       [-3.16314007e-03,  5.62129535e-02, -1.46675225e-04],
       [-3.19742094e-03,  5.67314625e-02, -2.84683324e-04],
       [-3.32227096e-03,  5.77824289e-02, -5.03044155e-04],
       [-3.81461000e-03,  6.04180222e-02, -8.53342447e-04],
       [-3.30515317e-03,  6.14570408e-02, -5.52792661e-20],
       [-3.26938848e-03,  6.16623237e-02, -1.26576672e-19],
       [-3.19311155e-03,  6.21486713e-02, -1.55914799e-19],
       [-3.07044682e-03,  6.31416924e-02, -8.94367023e-20],
       [-2.91647000e-03,  6.52717608e-02,  8.31194475e-20],
       [-2.63642778e-04,  6.85951194e-03,  5.48087554e-04],
       [ 6.37863656e-04, -2.54565973e-03

In [21]:
# import matplotlib.pyplot as plt
# %matplotlib inline
# section_number = np.arange(1, myvlm.n_spanwise+1, 1)

# fig = plt.figure()
# ax1 = fig.add_subplot(111)

# #ax1.plot(cl[0], section_number, label='jib')
# ax1.plot(cl[0],section_number, label='main')
# plt.ylabel('section number')
# plt.xlabel('section cl')
# plt.legend(loc='upper left')
# plt.show()